In [ ]:
import vectorbt as vbt

symbol = "BTC-USD"  # 交易对
#SOL,ETH,BTC
start_date = "2025-03-10"  # 开始日期
end_date = "2025-04-14"  # 结束日期
init_cash = 10000  # 初始资金

# 1. 获取数据（示例用雅虎数据，实际替换为你的数据源）
price = vbt.YFData.download(symbol, start=start_date, end=end_date,interval='15m').get("Close")
# 可选值：1m, 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 1wk, 1mo, 3mo

#注意：Yahoo Finance 的数据有一些限制：
#    - 分钟级数据只能获取最近7天
#    - 小时级数据可以获取近60天
#    - 日级及以上可以获取更长时间范围


In [2]:
price.head()

Datetime
2025-03-09 16:00:00+00:00    83433.750000
2025-03-09 16:15:00+00:00    83203.281250
2025-03-09 16:30:00+00:00    82807.617188
2025-03-09 16:45:00+00:00    82535.218750
2025-03-09 17:00:00+00:00    82743.242188
Freq: 15T, Name: Close, dtype: float64

In [3]:
 # 2. 定义策略规则（示例：双均线策略）
fast_ma = vbt.MA.run(price, window=10)
slow_ma = vbt.MA.run(price, window=20)


entries = fast_ma.ma_crossed_above(slow_ma)  # 金叉买入
exits = fast_ma.ma_crossed_below(slow_ma)    # 死叉卖出


In [ ]:
#数据的处理

In [ ]:
#梳理好因子

In [ ]:
#返回因子值


In [ ]:
print(entries[entries == True])

In [6]:
portfolio = vbt.Portfolio.from_signals(
    price, 
    entries, 
    exits,
    fees=0.001,  # 0.1%手续费
    init_cash=init_cash
)


In [7]:
stats = portfolio.stats()
key_metrics = {
        '总收益(%)': stats['Total Return [%]'],
        '收益(%)': stats['Total Return [%]'],
        '夏普比率': stats['Sharpe Ratio'],
        '胜率(%)': stats['Win Rate [%]'],
        '交易次数': stats['Total Trades'],
        '最大回撤(%)': stats['Max Drawdown [%]'],
    }
print("----------------------------------------")
print("初始资金", init_cash)
print("最终收益", portfolio.total_profit())
print("----------------------------------------")
for metric, value in key_metrics.items():
        print(f"{metric}: {value:.2f}{'%' if '%' in metric else ''}")

----------------------------------------
初始资金 10000
最终收益 -307.78128548241875
----------------------------------------
总收益(%): -3.08%
收益(%): -3.08%
夏普比率: -0.64
胜率(%): 32.58%
交易次数: 89.00
最大回撤(%): 15.44%


In [20]:
import plotly.graph_objects as go

# 计算布林带
bb = vbt.BBANDS.run(price, window=20, alpha=2)
rsi = vbt.RSI.run(price, window=14)

# 使用 vectorbt 的内置绘图方法
fig = portfolio.plot(
    subplots=['cum_returns', 'trade_pnl', 'orders'],
    make_subplots_kwargs=dict(
        rows=4,  # 指定总行数为4
        cols=1,
        shared_xaxes=True,
        vertical_spacing=0.05
    )
)

# 创建新的RSI子图
fig.add_trace(
    go.Scatter(
        x=price.index,
        y=rsi.rsi,
        name='RSI',
        line=dict(color='purple')
    ),
    row=4, col=1
)
# bb.plot(fig=fig)  # 直接在现有图表上添加布林带

# 添加RSI参考线
fig.add_hline(y=30, line_dash="dash", line_color="red", row=4)
fig.add_hline(y=70, line_dash="dash", line_color="red", row=4)

# 设置RSI的y轴范围
fig.update_yaxes(range=[0, 100], title_text="RSI", row=4, col=1)

fig.show()

In [60]:
trades = portfolio.trades

In [61]:
trades.records_readable

,Exit Trade Id,Column,Size,Entry Timestamp,Avg Entry Price,Entry Fees,Exit Timestamp,Avg Exit Price,Exit Fees,PnL,Return,Direction,Status,Position Id
0,0,0,0.120041,2025-03-09 18:25:00+00:00,83221.742188,9.990010,2025-03-09 19:20:00+00:00,82426.125000,9.894503,-115.391089,-0.011551,Long,Closed,0
1,1,0,0.119244,2025-03-09 20:05:00+00:00,82811.406250,9.874734,2025-03-09 21:20:00+00:00,82036.406250,9.782320,-112.070882,-0.011349,Long,Closed,1
2,2,0,0.120567,2025-03-10 00:10:00+00:00,80973.960938,9.762775,2025-03-10 00:55:00+00:00,81258.703125,9.797106,14.770587,0.001513,Long,Closed,2
3,3,0,0.119574,2025-03-10 01:00:00+00:00,81769.765625,9.777531,2025-03-10 03:30:00+00:00,82011.460938,9.806432,9.316492,0.000953,Long,Closed,3
4,4,0,0.118825,2025-03-10 04:15:00+00:00,82363.468750,9.786838,2025-03-10 05:50:00+00:00,82276.734375,9.776532,-29.869581,-0.003052,Long,Closed,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,247,0,0.079830,2025-04-11 06:05:00+00:00,81048.921875,6.470126,2025-04-11 07:35:00+00:00,81275.101562,6.488182,5.097591,0.000788,Long,Closed,247
248,248,0,0.079628,2025-04-11 07:45:00+00:00,81317.906250,6.475219,2025-04-11 11:30:00+00:00,82189.875000,6.544653,56.413650,0.008712,Long,Closed,248
249,249,0,0.079627,2025-04-11 13:05:00+00:00,82027.664062,6.531576,2025-04-11 13:15:00+00:00,81821.359375,6.515149,-29.474047,-0.004513,Long,Closed,249
250,250,0,0.078292,2025-04-11 13:40:00+00:00,83049.429688,6.502132,2025-04-11 14:35:00+00:00,81843.617188,6.407726,-107.315703,-0.016505,Long,Closed,250
